In [3]:
'''Задачи:
- вывести топ 20 понравившихся, комментируемых, репостнутых постов по количеству (для обоих датасетов с постами)
- вывести топ 20 пользователей по сделанным лайкам и репостам (для репостов используйте "copy_history", из датасета постов подписчиков)
- получить репосты исходных постов группы itmo (posts.json) из постов пользователей (результат должен быть похож на (group_post_id, Array (user_post_ids)) ) 
- найти смайлики в постах и комментариях к постам (негативные, позитивные, нейтральные) (можете использовать внешние библиотеки или предопределенные списки смайликов) (используйте функцию spark udf и broadcast для смайликов
- Найти друзей. Идея, если пользователи лайкают друг друга, то они друзья.  (*бонус)
- Найти поклонников. Идея, если пользователь лайкает другого, и это не взаимно, то первый поклонник. 
'''

'Задачи:\n- вывести топ 20 понравившихся, комментируемых, репостнутых постов по количеству (для обоих датасетов с постами)\n- вывести топ 20 пользователей по сделанным лайкам и репостам (для репостов используйте "copy_history", из датасета постов подписчиков)\n- получить репосты исходных постов группы itmo (posts.json) из постов пользователей (результат должен быть похож на (group_post_id, Array (user_post_ids)) ) \n- найти смайлики в постах и комментариях к постам (негативные, позитивные, нейтральные) (можете использовать внешние библиотеки или предопределенные списки смайликов) (используйте функцию spark udf и broadcast для смайликов\n- Найти друзей. Идея, если пользователи лайкают друг друга, то они друзья.  (*бонус)\n- Найти поклонников. Идея, если пользователь лайкает другого, и это не взаимно, то первый поклонник. \n'

In [1]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import emojis
import functools

In [2]:

os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'
#print(os.environ.get('JAVA_HOME'))

conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [4]:
big_data_dir = ''
followers = os.path.join(big_data_dir, 'followers.parquet/followers.parquet')
#concated with cat
followers_posts = os.path.join(big_data_dir, 'followers_posts_api_final.json/followers_posts_api_final_common.json')
followers_posts_likes = os.path.join(big_data_dir, 'followers_posts_likes.parquet/followers_posts_likes_1.parquet')
posts = os.path.join(big_data_dir, 'posts_api.json/posts_api.json')
posts_likes = os.path.join(big_data_dir, 'posts_likes.parquet/posts_likes.parquet')

In [3]:
from pyspark.sql.functions import col, isnan, when, trim, sort_array, explode, collect_list
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, LongType
import pandas as pd

### Task 1

In [11]:
posts_df = sqlContext.read.json(posts)

In [12]:
followers_posts_df = sqlContext.read.json(followers_posts)

In [13]:
followers_posts_df.columns

['attachments',
 'comments',
 'copy_history',
 'copyright',
 'date',
 'final_post',
 'from_id',
 'geo',
 'id',
 'is_pinned',
 'key',
 'likes',
 'owner_id',
 'post_source',
 'post_type',
 'reposts',
 'signer_id',
 'text',
 'unavailable',
 'views']

In [15]:
def get_top_posts(posts_df, prefix):
    posts_df = posts_df.na.drop(subset=["date"])

    for column, file_postfix in [('like', 'liked'), ('repost', 'reposted'), ('comment', 'commented')]:
        answer = posts_df.orderBy(posts_df[column + 's'].desc(), asc=False).limit(20).toPandas()
        answer_json = answer.to_json()
        with open('{}_top_{}.json'.format(prefix, file_postfix), 'w') as f:
            f.write(answer_json)

In [16]:
get_top_posts(posts_df, 'task1_posts')

In [17]:
get_top_posts(followers_posts_df, 'task1_followers_posts')

### Task 2

In [18]:
posts_likes_df = spark.read.load(posts_likes)

In [19]:
likes_count = posts_likes_df.groupby('likerId').count()
result = likes_count.orderBy(col('count').desc(), asc=False).limit(20)
result_json = result.toPandas().to_json()

In [20]:
with open('task2_top_followers_likes.json', 'w') as f:
    f.write(result_json)

In [21]:
df_history = followers_posts_df.withColumn('copy_history', explode('copy_history'))
result = df_history.filter(df_history.copy_history.owner_id == '-94').groupby('owner_id').count().sort(
    'count', ascending=False).limit(20)
result_json = result.toPandas().to_json()

In [39]:
with open('task2_top_followers_reposts.json', 'w') as f:
    f.write(result_json)

### Task 3

In [22]:
followers_df = spark.read.load(followers)

In [23]:
followers_posts_df.groupby('owner_id').count().orderBy(col('count').desc()).limit(20).show()

+---------+-----+
| owner_id|count|
+---------+-----+
|  2547211|40204|
|357231922|26715|
|168543860|18853|
| 50348231|11906|
| 25646344|11122|
|     null| 9858|
|176861294| 9033|
|141687240| 8808|
|445159771| 8704|
|143207077| 8471|
|194073434| 8146|
|    29840| 7293|
|524656784| 7263|
|459339006| 6814|
|514384760| 6578|
|483715951| 6140|
|412181460| 5813|
|461319529| 5724|
|451211328| 5651|
|426396104| 5545|
+---------+-----+



In [24]:
df_filter = df_history.filter(df_history.copy_history.owner_id == '-94')
df_select = df_filter.select(df_filter.copy_history.id.alias("group_post_id"), df_filter.id.alias("user_post_id"))
final = df_select.sort('copy_history.id')

In [25]:
#final.show()

In [46]:
result = final.groupby("group_post_id").agg(collect_list("user_post_id")).sort('group_post_id')

In [48]:
result_json = result.toPandas().to_json()
with open('task3_followers_reposts.json', 'w') as f:
    f.write(result_json)

### Task 4

In [10]:
from pyspark.sql.functions import *

In [7]:
posts_df = spark.read.json('posts_api.json/posts_api.json')
posts_df.createOrReplaceTempView("posts")

In [8]:
def extract_emoji(text):
    return(emojis.get(text))
spark.udf.register("extract_emoji", extract_emoji)

<function __main__.extract_emoji(text)>

In [11]:
emojis_df = spark.sql("SELECT key, extract_emoji(text) as emoji FROM posts WHERE text is not null")
emojis_df.select('key', 'emoji').sort(desc("emoji")).coalesce(1).write.format("json")

In [12]:
result = emojis_df.toPandas()

In [14]:
result_json = result.to_json()

In [15]:
with open('task4_post_emojis.json', 'w') as f:
    f.write(result_json)

### Task 5

In [4]:
followers_posts_likes_1 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_1.parquet")
followers_posts_likes_2 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_2.parquet")
followers_posts_likes_3 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_3.parquet")
followers_posts_likes_4 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_4.parquet")
followers_posts_likes_5 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_5.parquet")
followers_posts_likes_6 = \
    spark.read.load("followers_posts_likes.parquet/followers_posts_likes_6.parquet")

In [5]:
data_followers_posts_likes = functools.reduce(pyspark.sql.DataFrame.unionAll, 
                                              [followers_posts_likes_1, 
                                               followers_posts_likes_2, 
                                               followers_posts_likes_3, 
                                               followers_posts_likes_4, 
                                               followers_posts_likes_5, 
                                               followers_posts_likes_6])

In [8]:
data_followers_posts_likes = data_followers_posts_likes.select(pyspark.sql.functions.col('ownerId'), 
                                 pyspark.sql.functions.col('likerId')).\
    where(pyspark.sql.functions.col('ownerId') != pyspark.sql.functions.col('likerId')).distinct()

friends = data_followers_posts_likes.alias('t1').\
    join(data_followers_posts_likes.alias('t2'), 
         (pyspark.sql.functions.col('t1.ownerId') == pyspark.sql.functions.col('t2.likerId')) &
         (pyspark.sql.functions.col('t1.likerId') == pyspark.sql.functions.col('t2.ownerId'))).\
    select(pyspark.sql.functions.col('t1.ownerId').alias("id_user"), 
           pyspark.sql.functions.col('t2.ownerId').alias("id_friend")).\
    sort("id_friend").groupby("id_user").agg(pyspark.sql.functions.collect_list("id_friend")).sort("id_user")

In [ ]:
with open('task5_users_friends.json', 'w') as file:
    file.write(friends.toPandas().to_json())

### Task 6

In [7]:
fans = data_followers_posts_likes.alias('t1').\
    join(data_followers_posts_likes.alias('t2'), 
         (pyspark.sql.functions.col('t1.ownerId') == pyspark.sql.functions.col('t2.likerId')) &
         (pyspark.sql.functions.col('t1.likerId') == pyspark.sql.functions.col('t2.ownerId')), 
        "left_outer").where(pyspark.sql.functions.col('t2.likerId').isNull()).\
    select(pyspark.sql.functions.col('t1.ownerId').alias("id_user"), 
           pyspark.sql.functions.col('t1.likerId').alias("id_fan")).\
    sort("id_fan").groupby("id_user").agg(pyspark.sql.functions.collect_list("id_fan")).sort("id_user")

In [ ]:
with open('task6_users_fans.json', 'w') as file:
    file.write(fans.toPandas().to_json())